# Trabalho Algortimo Genético - Inteligência Artificial


Acadêmicos: Alani Rigotti de Oliveira, Gabriel Milke, Gabriel Silva e Thyago Karsten.

In [28]:
import numpy as np
import math
import random

CIDADES = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

CIDADES_X = [0.77687122, 0.55726533, 0.65639441, 0.60439895, 0.10984792,
            0.30681839, 0.03642046, 0.50750194, 0.79819788, 0.79896875,
            0.14326940, 0.07110193, 0.72613150, 0.22624105, 0.62480412,
            0.54832279, 0.39699388, 0.07545496, 0.67595097, 0.07429705]

CIDADES_Y = [0.27943920, 0.11661366, 0.39053913, 0.66616904, 0.69857584,
            0.20730006, 0.50247213, 0.07393869, 0.67991802, 0.39749278,
            0.14151256, 0.12773617, 0.37197290, 0.69033435, 0.91890348,
            0.52333815, 0.42525695, 0.37166915, 0.99033329, 0.15694232]

NUMERO_INDIVIDUOS = 20
NUMERO_GERACOES = 10000
TAXA_DE_MUTACAO = 5

# Valores utilizado para calcular a função de aptidão
max = sum(d for d in range(1, NUMERO_INDIVIDUOS + 1))
# Para não precisarmos calular as probabilidades todas as vezes, a lista será ordenada ao invés de alterar o cálculo de prob.
probabilidades = [(NUMERO_INDIVIDUOS - d)/max for d in range(0, NUMERO_INDIVIDUOS)]

def main():
  global individuos
  individuos = inicializarIndividuos()

  for _ in range(NUMERO_GERACOES):

    individuos = gerarNovaGeracao(individuos, calcularDistancias(individuos))
    criar_descendentes()

  distancias_finais = calcularDistancias(individuos)
  print(distancias_finais)

  distancias_finais.sort(key=lambda x: x[1])

  qtd_cromossomos = len(individuos)
  qtd_cidades = len(CIDADES)
  melhor_custo = distancias_finais[0][1]
  melhor_solucao = individuos[distancias_finais[0][0]]

  print('Tamanho da População:', qtd_cromossomos)
  print('Taxa de Mutação:', TAXA_DE_MUTACAO)
  print('Número de Cidades:', qtd_cidades)
  print('Melhor Custo:', melhor_custo)
  print('Melhor Solução:', melhor_solucao)

def inicializarIndividuos():

  individuos = []

  for _ in range(NUMERO_INDIVIDUOS):
    random.shuffle(CIDADES)
    individuos.append(CIDADES.copy())

  return individuos

def calcularDistancias(individuos: list):

  distancias_totais = []

  # Os indivíduos possuem uma lista com a ordem das cidades
  for [index_ind, individuo] in enumerate(individuos):
    distancia = 0
    for index_cid in range(len(individuos[index_ind])):
      if index_cid != 0:

        # Ambos os indexes tem seu valor por conta da lista começar em 1 e não 0
        cidade_atual = CIDADES[individuo[index_cid] - 1]
        cidade_anterior = CIDADES[individuo[index_cid - 1] - 1]

        # Para calcular a distancia entre dois pontos num plano:
        # d = √(x1 - x2)²  + (y1 - y2)²
        distancia += math.sqrt(math.pow(CIDADES_X[cidade_atual - 1] - CIDADES_X[cidade_anterior - 1], 2) + math.pow(CIDADES_Y[cidade_atual - 1] - CIDADES_Y[cidade_anterior - 1], 2))

    distancias_totais.append((index_ind, distancia))

  return distancias_totais


def gerarNovaGeracao(individuos: list, distancias: list):

  # Ordena a lista de distancias
  distancias.sort(key= lambda dist : dist[1])

  novos_individuos = []

  for _ in range(NUMERO_INDIVIDUOS):
    novos_individuos.append(gerarFilho(escolherPais(individuos, distancias)))

  return novos_individuos

def gerarFilho(pais):

  filho = []

  for cidade in pais[0]:
    filho.append(cidade) if np.random.choice([0,1]) == 1 else filho.append(None)

  cidades_faltando = list(filter(lambda c : (not c in filho), CIDADES))

  for [index, cidade] in enumerate(filho):
    if cidade == None:
      filho[index] = encontrarPrimeiraCidadeFaltando(cidades_faltando, pais[1])

  return filho

def escolherPais(individuos: list, distancias: list):

  # O calculo de peso se dará da seguinte forma:
  # Peso dos itens:
  #  0 -> 20
  #  1 -> 19 ...
  # 19 -> 1

  # Será escolhido um valor com base na lista ordenada de distâncias.
  # Utilizamos esse valor para retornar o objeto completo do indivíduo

  index_pai1 = distancias[np.random.choice(len(individuos), p=probabilidades)][0]

  # Garante que os dois pais não serão o mesmo
  while True:
    index_pai2 = distancias[np.random.choice(len(individuos), p=probabilidades)][0]
    if index_pai1 != index_pai2:
      break

  return (individuos[index_pai1], individuos[index_pai2])


def encontrarPrimeiraCidadeFaltando(cidades_faltando, pai):
  for cidade in pai:
    if cidade in cidades_faltando:
      cidades_faltando.remove(cidade)
      return cidade

def mutacao(descendente):
  num_random = random.randint(1, 100)
  if num_random <= TAXA_DE_MUTACAO:
    cidade_1 = random.randint(0, 19)
    while True:
      cidade_2 = random.randint(0, 19)
      if cidade_1 != cidade_2:
        break
    descendente[cidade_1], descendente[cidade_2] = descendente[cidade_2], descendente[cidade_1]

def cycle(descendente_1, descendente_2, posicao_troca):
  trocar_valores_descendentes(descendente_1, descendente_2, posicao_troca)
  posicao_valor_repetido = validar_se_repete_valor(descendente_1, posicao_troca)
  if posicao_valor_repetido != -1:
    cycle(descendente_1, descendente_2, posicao_valor_repetido)
  else:
    posicao_valor_repetido = validar_se_repete_valor(descendente_2, posicao_troca)
    if posicao_valor_repetido != -1:
      cycle(descendente_1, descendente_2, posicao_valor_repetido)

def trocar_valores_descendentes(descendente_1, descendente_2, posicao):
  descendente_1[posicao], descendente_2[posicao] = descendente_2[posicao], descendente_1[posicao]

def validar_se_repete_valor(array, posicao):
  valor_posicao = array[posicao]
  posicao_valor_repetido = -1
  for i, valor in enumerate(array):
    if valor == valor_posicao and i != posicao:
      posicao_valor_repetido = i
      break
  return posicao_valor_repetido

def criar_descendentes():
    global individuos
    novos_individuos = individuos.copy()
    for i in range(0, 10, 2):
        posicao_randomica = random.randint(0, 19)
        descendente_1 = novos_individuos[i].copy()
        descendente_2 = novos_individuos[i + 1].copy()
        cycle(descendente_1, descendente_2, posicao_randomica)
        mutacao(descendente_1)
        mutacao(descendente_2)
        individuos[i] = descendente_1
        individuos[i + 1] = descendente_2

if __name__ == '__main__':
  main()

[(0, 3.743629735972757), (1, 3.743629735972757), (2, 3.743629735972757), (3, 3.743629735972757), (4, 3.743629735972757), (5, 3.743629735972757), (6, 3.743629735972757), (7, 3.743629735972757), (8, 3.743629735972757), (9, 3.743629735972757), (10, 3.743629735972757), (11, 3.743629735972757), (12, 3.743629735972757), (13, 3.743629735972757), (14, 3.743629735972757), (15, 3.743629735972757), (16, 3.743629735972757), (17, 3.743629735972757), (18, 3.743629735972757), (19, 3.743629735972757)]
Tamanho da População: 20
Taxa de Mutação: 5
Número de Cidades: 20
Melhor Custo: 3.743629735972757
Melhor Solução: [15, 13, 11, 16, 9, 20, 3, 4, 19, 7, 6, 5, 18, 10, 14, 17, 2, 12, 1, 8]
